__Step 1__: Launch a **t2.medium** instance (*redhat 7.x*)

__Step 2__: login and run the following commands

* ``` pushd /tmp ```
* ``` sudo yum install wget -y # to get files we need ```

* ``` wget https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm # the epel repo for dockr-compose```
* ``` sudo yum install -y epel-release-latest-7.noarch.rpm ```
* ``` popd ```

* ``` sudo yum update -y ```
* ``` sudo yum install vim-enhanced mlocate python-pip python-wheel python-wheel postgresql-devel wget tmux git -y ```

* ``` sudo sh -c "wget -qO- https://get.docker.com/ | sh" # installs docker ```
* ``` sudo groupadd -f docker && sudo usermod -aG docker $USER # creates docker user ```


__Step 3__: Logout and login back. (due to setting a new user group for docker)

__Step4__: install docker-compose 
* ``` sudo systemctl enable docker.service ```
* ``` sudo systemctl start docker.service ```
* ``` sudo pip install docker-compose # install docker compose now we have docker ```
* ``` alias dc='docker-compose' ```
* ``` echo "alias dc='docker-compose'" > ~/.bashrc ```
* ``` echo "alias dc='docker-compose'" > ~/.bash_profile ```

__step 5__: Create DART environment files to run using docker-compose.

* ``` cd ~ ```
* ``` mv dart/tools/vagrant . ```
* ``` rm -rf dart ```
* ``` cd vagrant  ```
* ``` git clone https://github.com/RetailMeNotSandbox/dart.git # this source code will be referenced by d-compose```
* ``` sudo pip install -r tools_requirements.txt # installs jinja so we can create d-compose files```
* ``` python generate_deployment_files.py # generates files from templates, a set for each part (engine, web, ...) ```
* ``` cd docker_files ```
* ``` sed -i 's|/vagrant_data/docker_files|.|g' docker-compose.yml ```
* ``` sed -i 's|/vagrant_data|.|g' docker-compose.yml # removing prefixes since we will use the files in cur dir```
* ``` sed -i 's|/docker_files|.|g' docker-compose.yml ```
* ``` git clone https://github.com/RetailMeNotSandbox/dart.git # the source code```
* ``` docker-compose up -d --build web postgres elasticmq # launch dependent containers first so we can init dart ```

__Step 6__: init DART database
``` 
STOP=1;
while [ $STOP -gt 0 ]; do
  sh -c 'docker-compose logs web | tail -10'
  docker-compose exec web curl -XPOST "http://127.0.0.1:5000/admin/create_all";
  STOP=$?;
  echo "Waiting for web worker to complete. Trying again in 15 seconds ..."
  sleep 15;
done 
```

__step 7__: Launch the remaining DART components.
* ``` docker-compose up -d --build engine trigger subscription ```
* ``` docker-compose ps # to verify ```

__Step 8__: Add no-op engine so we can run unit tests.
```
STOP=1; 
while [ $STOP -gt 0 ]; do 
  docker-compose exec web python /tmp/src/python/dart/engine/no_op/add_engine.py; 
  STOP=$?; 
  sleep 1; 
done
```

__step 9__: Run unit tests.
* ``` docker-compose exec web python -m unittest discover /tmp/src/python/dart/test/schema/ "test_*.py" ```
* ``` docker-compose exec web python -m unittest discover /tmp/src/python/dart/test/graph "test_*.py" ```
* ``` docker-compose exec web python -m unittest discover /tmp/src/python/dart/test/crud "test_*.py" ```

_Step 10__: Misc.
* Open port 5000 as a custom tcp rule (use `my ip`) then you can access this instance via browser
* open 5432 for sql and so on ..(see docker-compose file for ports)